<a href="https://colab.research.google.com/github/sachinHansak/RAG-Application-with-Llama2-LlamaIndex-/blob/main/Llama_2_with_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
## Embedding
!pip install install sentence_transformers

In [ ]:
!pip install llama-index

In [ ]:
%pip install llama-index-llms-huggingface

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
#from llama_index.core.prompts import LangchainPromptTemplate
from llama_index.core import PromptTemplate

In [ ]:
documents=SimpleDirectoryReader("/content/drive/MyDrive/data_researches").load_data()
documents

In [ ]:
# template = (
#     "You are a Q&A assistant. Your goal is to answer questions as\n"
#     "accurately as possible based on the instructions and context provided.\n"
#     "---------------------\n"
#     "{context_str}"
#     "\n---------------------\n"
#     "Given this information, please answer the question: {query_str}\n"
# )
# qa_template = PromptTemplate(template)

# # you can create text prompt (for completion API)
# prompt = qa_template.format(context_str=..., query_str=...)

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
%pip install llama-index-embeddings-langchain

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-15-f6dd2f399444>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [ ]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7faf4e3ee440>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7faf4e3ee440>, id_func=<function default_id_func at 0x7fb01e4e2560>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7faf4c26abf0>, callback_manager=<llama_index.core.callbacks.ba

In [ ]:
index=VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
    )

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is attention is all you need?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

Attention is a powerful tool for NLP tasks, but it is not the only thing you need to achieve good results. While attention mechanisms can help the model focus on relevant parts of the input, they do not provide any guarantees of accuracy or completeness. Other factors such as the choice of model architecture, the quality of the training data, and the optimization algorithms used can also have a significant impact on the performance of the model. Therefore, it is important to consider attention as one of many tools in the NLP toolkit, and to evaluate its effectiveness in the context of the specific task and dataset being used.


In [ ]:
response=query_engine.query("what is YOLO?")

In [ ]:
print(response)

YOLO stands for You Only Look Once. It is a real-time object detection system that is fast, simple, and accurate. YOLO uses a single neural network to predict bounding boxes and class probabilities directly from full images. It is trained on full images and directly optimizes detection performance, making it extremely fast and achieving more than twice the mean average precision of other real-time systems. YOLO is refreshingly simple, with a single convolutional network simultaneously predicting multiple bounding boxes and class probabilities for those boxes. It is a unified model that has several benefits over traditional methods of object detection, including its ability to process streaming video in real-time with less than 25 milliseconds of latency.


In [ ]:
response=query_engine.query("How YOLO is working?")

In [ ]:
print(response)
re = str(response)

YOLO is a real-time object detection system that uses a single neural network to predict bounding boxes and class probabilities directly from full images. The network is trained on full images and optimizes detection performance directly. YOLO is simple and fast, with a base network running at 45 frames per second on a Titan X GPU and a fast version running at more than 150 fps. YOLO achieves more than twice the mean average precision of other real-time systems and is able to process streaming video in real-time with less than 25 milliseconds of latency. YOLO is trained on full images and directly optimizes detection performance, which allows it to maintain high performance on artwork.


In [ ]:
import textwrap
print(textwrap.fill(re, width=100))

YOLO is a real-time object detection system that uses a single neural network to predict bounding
boxes and class probabilities directly from full images. The network is trained on full images and
optimizes detection performance directly. YOLO is simple and fast, with a base network running at 45
frames per second on a Titan X GPU and a fast version running at more than 150 fps. YOLO achieves
more than twice the mean average precision of other real-time systems and is able to process
streaming video in real-time with less than 25 milliseconds of latency. YOLO is trained on full
images and directly optimizes detection performance, which allows it to maintain high performance on
artwork.


In [ ]:
response=query_engine.query("what is the multi headed attention?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
re = str(response)
import textwrap
print(textwrap.fill(re, width=100))

The multi-headed attention is a mechanism in the Transformer architecture that allows the model to
jointly attend to information from different representation subspaces at different positions. In
other words, the model can attend to different parts of the input sequence simultaneously, which
helps it capture longer-range dependencies and better understand the context in which a word or
phrase appears. This is achieved by dividing the attention mechanism into multiple attention heads,
each of which computes its own attention weights and then combines the results. The idea is that
different attention heads will attend to different aspects of the input, and by combining their
outputs, the model can capture a richer representation of the input.
